In [57]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

In [58]:
df = pd.read_csv('cbcls.csv')
df = df.dropna(thresh = 15)
descriptions = df.iloc[0]
df = df.iloc[1:]

print(len(df))

C:\Users\Matth\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (1,2,3,7,10,11,13,14,15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,39,41,42,43,45,46,47,49,50,51,53,54,55,57,58,59,61,62,63,65,66,67,69,70,71,73,74,75,77,78,79,81,82,83,85,86,87,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


37293


In [59]:
print(descriptions)

Unnamed: 0                                                                 0
collection_id                                                  collection_id
abcd_cbcls01_id                                              abcd_cbcls01_id
dataset_id                                                        dataset_id
subjectkey                 The NDAR Global Unique Identifier (GUID) for r...
                                                 ...                        
cbcl_scr_07_stress_nm      Stress CBCL Scale2007 Scale (number of missing...
cbcl_scr_07_stress_nm_2    Stress CBCL Scale2007 Scale (number of missing...
collection_title                                            collection_title
study_cohort_name                                          study_cohort_name
adhd                                                                       0
Name: 0, Length: 94, dtype: object


In [60]:
X = df.iloc[:,7:90:4]
y = df['adhd']

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [63]:
model = LogisticRegression(class_weight={0:1266,1:9149})
model.fit(X_train,y_train)

C:\Users\Matth\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight={0: 1266, 1: 9149})

In [64]:
importance = model.coef_[0]

coefficients = []
for i in range(len(importance)):
    coefficients.append((X.columns[i],importance[i]))
    
to_show = 5  

sorted_coef = sorted(coefficients,key=lambda tup:abs(tup[1]))
bottom_5 = sorted_coef[:to_show]
top_5 = sorted_coef[-to_show:]

print(f'*** The {to_show} least predictive measures were (odds ratio): ***')
for index,(label,value) in enumerate(bottom_5):
    print(f"{len(coefficients)-index}. {descriptions[label].split(' / ')[0]}: {round(np.exp(value),3)}")
print(f'\n*** The {to_show} most predictive measures were (odds ratio): ***')
for index,(label,value) in enumerate(top_5):
    print(f"{to_show-index}. {descriptions[label].split('. / ')[0]}: {round(np.exp(value),3)}")

*** The 5 least predictive measures were (odds ratio): ***
21. Age in months at the time of the interview/test/sampling/imaging.: 0.997
20. Thought CBCL Syndrome Scale (t-score): 0.996
19. Somatic CBCL Syndrome Scale (t-score): 0.99
18. AnxDisord CBCL DSM5 Scale (t-score): 0.987
17. Opposit CBCL DSM5 Scale (t-score): 1.016

*** The 5 most predictive measures were (odds ratio): ***
5. External CBCL Syndrome Scale (t-score): 0.95
4. Stress CBCL Scale2007 Scale (t-score): 1.059
3. Sluggish Cognitive Tempo (SCT) CBCL Scale2007 Scale (t-score): 0.944
2. Attention CBCL Syndrome Scale (t-score): 1.099
1. TotProb CBCL Syndrome Scale (t-score): 1.196


In [65]:
score = model.score(X_test,y_test)
print(score)

0.7849577691379541
